In [1]:
import numpy as np
from library.cnn_model import CNN
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import  ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras import Model
import matplotlib.pyplot as plt
from library.imagedataprocessor import ImageDataProcessor
import os

In [2]:
finance = "XLE"

In [3]:
# Define la ruta raíz desde donde comenzar a recorrer las carpetas
root_path = f'C:/Users/Usuario/Desktop/Maestria/FinTech/CNN2/data/{finance}'

In [4]:
# Recorre las carpetas y archivos en el path definido
for root, dirs, files in os.walk(f'{root_path}/dataset'):
    # Imprimir los nombres de todas las carpetas en el directorio actual
    dirs[:] = [d for d in dirs if d not in ('training', 'testing')]
    dirs.sort()
    for dir_name in dirs:
        training = f'{os.path.join(root, dir_name)}/training'
        testing = f'{os.path.join(root, dir_name)}/testing/images'
        os.makedirs(training, exist_ok=True)
        os.makedirs(testing, exist_ok=True)
        
        # Ejemplo de uso:
        csv_training_path = f'{os.path.join(root, dir_name)}/data_training_transformed.csv'
        csv_testing_path = f'{os.path.join(root, dir_name)}/data_testing_transformed.csv'

        root_path_training = f'{os.path.join(root, dir_name)}/training/'
        root_path_testing = f'{os.path.join(root, dir_name)}/testing/images'

        # Crear una instancia de la clase
        processor_training = ImageDataProcessor(csv_training_path, root_path_training,  handle_imbalance=False)

        # Crear una instancia de la clase
        processor_testing = ImageDataProcessor(csv_testing_path, root_path_testing, is_testing=True)

        # Ejecutar el procesamiento completo
        processor_training.process()
        processor_testing.process()

In [5]:
temp = CNN()
temp.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

# Entrenamiento Modelo por todas las carpetas
for root, dirs, files in os.walk(f'{root_path}/dataset'):
    # Imprimir los nombres de todas las carpetas en el directorio actual
    dirs[:] = [d for d in dirs if d not in ('training', 'testing')]
    dirs.sort()
    for dir_name in dirs:       
        model_path = f'data/{finance}/Model_1.keras'
        print(f'{os.path.join(root, dir_name)}')
        # Verificar si el archivo del modelo existe
        if os.path.exists(model_path):
            # Cargar el modelo si existe
            temp = load_model(model_path, custom_objects={'CNN': CNN})   
        else:
            learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.75, 
                                            min_lr=0.0001)

            callbacks = [learning_rate_reduction]        
            image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.25)

        training_dataset = image_generator.flow_from_directory(directory = f'{os.path.join(root, dir_name)}/training', 
                                                            target_size=(15,15),
                                                            batch_size = 16, 
                                                            color_mode='grayscale',                                                     
                                                            subset="training",
                                                            class_mode='categorical')
                                          

        validation_dataset = image_generator.flow_from_directory(directory = f'{os.path.join(root, dir_name)}/training', 
                                                                target_size=(15,15),
                                                                batch_size = 16, 
                                                                color_mode='grayscale',   
                                                                subset="validation",
                                                                class_mode='categorical')
        
        history = temp.fit(training_dataset, 
                           steps_per_epoch=100, 
                           epochs=40,
                           validation_data=validation_dataset, 
                           validation_batch_size =50, 
                           callbacks = [learning_rate_reduction])
        
        image_generator_testing = ImageDataGenerator(rescale=1/255)
        
        testing_dataset = image_generator_testing.flow_from_directory(directory=f'{os.path.join(root, dir_name)}/testing',
                                                                    target_size=(15,15),
                                                                    batch_size = 32, 
                                                                    color_mode='grayscale')
        
        result = temp.predict(testing_dataset)
        max_index_1 = np.argmax(result, axis=1)
        result_csv = pd.read_csv(f'{os.path.join(root, dir_name)}/testing/images/cnn_result.csv')
        
        result_csv['Predicted'] = max_index_1.tolist()
        result_csv.to_csv(f'{os.path.join(root, dir_name)}/testing/images/cnn_result.csv')
        
        temp.save(f'{root_path}/Model_1.keras')


C:/Users/Usuario/Desktop/Maestria/FinTech/CNN2/data/XLE/dataset\dataset001
Found 717 images belonging to 3 classes.
Found 237 images belonging to 3 classes.
Epoch 1/40
  1/100 ━━━━━━━━━━━━━━━━━━━━ 3:42 2s/step - accuracy: 0.4375 - loss: 1.1084

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 44/100 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4158 - loss: 1.0971

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.4391 - loss: 1.0826 - val_accuracy: 0.3207 - val_loss: 1.0998 - learning_rate: 0.0010
Epoch 2/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5063 - loss: 1.0174 - val_accuracy: 0.5570 - val_loss: 1.0026 - learning_rate: 0.0010
Epoch 3/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5468 - loss: 0.9763 - val_accuracy: 0.5485 - val_loss: 1.0077 - learning_rate: 0.0010
Epoch 4/40
 42/100 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5681 - loss: 0.9359
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0007500000356230885.
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5655 - loss: 0.9436 - val_accuracy: 0.4937 - val_loss: 0.9862 - learning_rate: 0.0010
Epoch 5/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5722 - loss: 0.9385 - val_accuracy: 0.6118 - val_loss: 0.9557 - learning_rate: 7.5000e-04
Epoch 6/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5709 - loss: 0.9264 - val_ac

...

100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.6715 - loss: 0.8055 - val_accuracy: 0.5833 - val_loss: 0.8382 - learning_rate: 1.0000e-04
Epoch 2/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6704 - loss: 0.7837 - val_accuracy: 0.5733 - val_loss: 0.8511 - learning_rate: 1.0000e-04
Epoch 3/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6714 - loss: 0.7736 - val_accuracy: 0.5900 - val_loss: 0.8674 - learning_rate: 1.0000e-04
Epoch 4/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6848 - loss: 0.7475 - val_accuracy: 0.5833 - val_loss: 0.8843 - learning_rate: 1.0000e-04
Epoch 5/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6890 - loss: 0.7328 - val_accuracy: 0.6033 - val_loss: 0.8643 - learning_rate: 1.0000e-04
Epoch 6/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.6741 - loss: 0.7615 - val_accuracy: 0.5533 - val_loss: 0.8898 - learning_rate: 1.0000e-04
Epoch 7/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.70

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 55/100 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.5881 - loss: 0.9389

c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.5826 - loss: 0.9619 - val_accuracy: 0.4716 - val_loss: 1.1191 - learning_rate: 1.0000e-04
Epoch 2/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5656 - loss: 0.9437 - val_accuracy: 0.5217 - val_loss: 0.8995 - learning_rate: 1.0000e-04
Epoch 3/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5815 - loss: 0.9111 - val_accuracy: 0.5786 - val_loss: 0.9308 - learning_rate: 1.0000e-04
Epoch 4/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6056 - loss: 0.8841 - val_accuracy: 0.4983 - val_loss: 1.0206 - learning_rate: 1.0000e-04
Epoch 5/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5975 - loss: 0.8557 - val_accuracy: 0.5585 - val_loss: 0.9552 - learning_rate: 1.0000e-04
Epoch 6/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.6173 - loss: 0.8256 - val_accuracy: 0.5886 - val_loss: 0.9458 - learning_rate: 1.0000e-04
Epoch 7/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.63